In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time

def login(driver, site_info):
    """로그인 수행"""
    try:
        driver.get(site_info["login_url"])
        print(f"로그인 페이지 이동: {site_info['login_url']}")

        # ID 입력
        id_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, site_info["id_selector"]))
        )
        id_input.send_keys(site_info["id"])

        # PW 입력
        pw_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, site_info["pw_selector"]))
        )
        pw_input.send_keys(site_info["pw"])

        # 로그인 버튼 클릭
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, site_info["login_button_selector"]))
        )
        login_button.click()

        time.sleep(3)  # 로그인 후 페이지 로딩 대기

        print("✅ 로그인 성공!")
        return True

    except Exception as e:
        print(f"❌ 로그인 중 오류 발생: {e}")
        return False


def navigate_to_order_detail(driver, site_info):
    """주문배송조회 페이지 이동 → 첫 번째 주문번호 클릭 → 상세 페이지에서 정보 수집"""
    try:
        print("\n------------------")
        driver.get("https://www.dometopia.com/mypage/order_catalog")
        time.sleep(3)  # 페이지 로딩 대기

        print("✅ 주문배송조회 페이지 이동 완료!")

        # 주문번호 목록 가져오기
        order_numbers = driver.find_elements(By.CSS_SELECTOR, 
            "#mypage-order_catalog > div.doto-table-wrap > table > tbody > tr > td.order-number > a > strong")

        if not order_numbers:
            print("⚠️ 주문번호를 찾을 수 없음!")
            return False

        print("\n✅ 수집된 주문번호 목록:")
        for idx, order in enumerate(order_numbers, 1):
            print(f"{idx}. {order.text}")

        # 첫 번째 주문번호 클릭하여 상세 페이지 이동
        print(f"\n🔗 첫 번째 주문번호 ({order_numbers[0].text}) 클릭하여 상세 페이지로 이동...")
        order_numbers[0].click()
        time.sleep(3)  # 이동 후 대기

        print("✅ 이동 완료!")
        print("현재 URL:", driver.current_url)

        # 상세 페이지에서 정보 수집
        extract_order_details(driver, site_info)

        return True

    except Exception as e:
        print(f"❌ 주문 상세 페이지 이동 중 오류 발생: {e}")
        return False


def extract_order_details(driver, site_info):
    """주문 상세 페이지에서 배송번호, 배송사, 받는 사람 이름 수집"""
    try:
        print("\n📦 주문 상세 정보 수집 시작...")

        # 배송번호
        try:
            delivery_number = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, site_info["delivery_number_selector"]))
            ).text
        except:
            delivery_number = "정보 없음"

        # 배송사
        try:
            delivery_company = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, site_info["delivery_company_selector"]))
            ).text
        except:
            delivery_company = "정보 없음"

        # 받는 사람 이름
        try:
            recipient_name = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, site_info["name_selector"]))
            ).text
        except:
            recipient_name = "정보 없음"

        print("\n✅ 수집된 주문 상세 정보:")
        print(f"📦 배송번호: {delivery_number}")
        print(f"🚚 배송사: {delivery_company}")
        print(f"👤 받는 사람: {recipient_name}")

    except Exception as e:
        print(f"❌ 주문 상세 정보 수집 중 오류 발생: {e}")


if __name__ == "__main__":
    # JSON 파일 로드
    json_file_path = r"C:\Users\jh\택배송장\site_configs\dmtopia_config.json"
    with open(json_file_path, "r", encoding="utf-8") as f:
        site_info = json.load(f)

    # Chrome 옵션 설정
    options = webdriver.ChromeOptions()
    options.add_argument("--user-data-dir=C:\\selenium\\chrome-data")  
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Whale/4.30.291.11 Safari/537.36")

    # WebDriver 실행
    driver = webdriver.Chrome(options=options)

    # 로그인 및 주문번호 수집 → 첫 번째 주문번호 상세페이지 이동 → 배송정보 수집 실행
    if login(driver, site_info):
        if navigate_to_order_detail(driver, site_info):
            print("\n✅ 모든 작업 완료!")

    # WebDriver 종료
    driver.quit()


로그인 페이지 이동: https://www.dometopia.com/member/login
✅ 로그인 성공!

------------------
✅ 주문배송조회 페이지 이동 완료!

✅ 수집된 주문번호 목록:
1. 2025022111320119019
2. 2025022022374622291
3. 2025022014522420139
4. 2025022001335717571
5. 2025022001311317569
6. 2025022001293717568
7. 2025021917102322414
8. 2025021917081022411
9. 2025021917022322391
10. 2025021808173317627

🔗 첫 번째 주문번호 (2025022111320119019) 클릭하여 상세 페이지로 이동...
✅ 이동 완료!
현재 URL: https://www.dometopia.com/mypage/order_view?no=2025022111320119019

📦 주문 상세 정보 수집 시작...

✅ 수집된 주문 상세 정보:
📦 배송번호: 정보 없음
🚚 배송사: 정보 없음
👤 받는 사람: 김해정님에게 보내는 티켓 및 발송 정보 택배 (선불 또는 착불)

✅ 모든 작업 완료!
